In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
#import datawig

In [ ]:
df = pd.read_csv('../input/jobathon-analytics-vidhya/train.csv')
test = pd.read_csv('../input/jobathon-analytics-vidhya/test.csv')
print(df.shape,test.shape)

In [ ]:
df.head()

In [ ]:
test.head(1)

In [ ]:
df.info()

In [ ]:
test.info()

In [ ]:
#categorizing features
num_attribs = ['ID','Region_Code','Upper_Age', 'Lower_Age','Reco_Policy_Premium']
cat_attribs = ['City_Code','Accomodation_Type','Reco_Insurance_Type',
               'Is_Spouse','Health Indicator','Holding_Policy_Duration','Holding_Policy_Type','Reco_Policy_Cat',]
target = ['Response']
len(df.columns) == len(num_attribs)+len(cat_attribs)+len(target)

## EDA Part1

In [ ]:
df.describe()

In [ ]:
fig = df[num_attribs].hist(figsize=(20,15))
# can combine upper age and lower age

In [ ]:
fig = df[['Reco_Policy_Premium']].boxplot(figsize=(8,5))
#presence of Outliers

In [ ]:
fig = df[['Upper_Age','Lower_Age']].boxplot(figsize=(8,5))

In [ ]:
sns.countplot(x='Response',data=df)
#Imbalanced dataset

## baseline model

In [ ]:
df['Response'].value_counts()

In [ ]:
num_attribs1 = [i for i in num_attribs if i not in ['Upper_Age', 'Lower_Age']]
print(num_attribs)
print(num_attribs1)

In [ ]:
# Baseline Model

def compare_model(data,testing):
    
    num_attribs = ['Region_Code','Upper_Age', 'Lower_Age','Reco_Policy_Premium']
    cat_attribs = ['City_Code','Accomodation_Type','Reco_Insurance_Type',
               'Is_Spouse','Health Indicator','Holding_Policy_Duration','Holding_Policy_Type','Reco_Policy_Cat']
    target = ['Response']
    
    if testing==1:
        #testing part
        new_features_num = []
        new_features_cat = []
        drop_features = []
        num_attribs = [i for i in num_attribs if i not in drop_features]
        cat_attribs = [i for i in cat_attribs if i not in drop_features]
        num_attribs = num_attribs + new_features_num
        cat_attribs = cat_attribs + new_features_cat
        data = data.drop(drop_features,axis=1)
    
    #preprocessing
    
    imputer  = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    data['Health Indicator'] = imputer.fit_transform(data['Health Indicator'].values.reshape(-1,1))
    data['Holding_Policy_Duration'] = imputer.fit_transform(data['Holding_Policy_Duration'].values.reshape(-1,1))
    data['Holding_Policy_Type'] = imputer.fit_transform(data['Holding_Policy_Type'].values.reshape(-1,1))
    
    
    print("num attribs :{}".format(num_attribs))
    print("cat attribs :{}".format(cat_attribs))
    
    
    X=data[num_attribs+cat_attribs]
    
    y=data[target]
    
    print('Data shape {} X shape {} y shape {} '.format(data.shape,X.shape,y.shape))

    full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(), cat_attribs)
    ],remainder='passthrough')
    

    X = full_pipeline.fit_transform(X)
    
    #Train-Test split
    print("X shape {} and y shape {}".format(X.shape,y.shape))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)
    print('X_train shape {} X_test shape {} y_train shape {} y_test shape {}'.format(X_train.shape,X_test.shape,y_train.shape,y_test.shape))
    
    #Model1
    dc = DecisionTreeClassifier(random_state=0)
    dc.fit(X_train,y_train)
    y_pred_dc = dc.predict(X_test)
    dc_acc = accuracy_score(y_test, y_pred_dc, normalize=True)
    roc_auc_dc = roc_auc_score(y_test, dc.predict_proba(X_test)[:, 1])
    
    #Model2
    rf = RandomForestClassifier(random_state=0)
    rf.fit(X_train,y_train)
    y_pred_rf = rf.predict(X_test)
    rf_acc = accuracy_score(y_test, y_pred_rf, normalize=True)
    roc_auc_rf = roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])
    
    #Model3
    lr = LogisticRegression(random_state=0).fit(X, y)
    y_pred_lr = lr.predict(X_test)
    lr_acc = accuracy_score(y_test, y_pred_lr, normalize=True)
    roc_auc_lr = roc_auc_score(y_test, lr.decision_function(X_test))
    
    
    print('Tree accuracy: {} , Forest accuracy:{} , Log Reg accuracy:{}'.format(round(dc_acc,4),round(rf_acc,4),round(lr_acc,4)))
    print('Tree roc_auc: {} , Forest roc_auc:{} , Log Reg roc_auc:{}'.format(round(roc_auc_dc,4),round(roc_auc_rf,4),round(roc_auc_lr,4))) 

In [ ]:
compare_model(df,0)

## EDA part2

In [ ]:
cat_attribs

In [ ]:
k=0
for i in cat_attribs:
    print('Column : {}'.format(i))
    print(df[i].value_counts())
    print('**'*50)
    k+=1
print("Total {} category columns are there".format(k))

#

In [ ]:
def percent(axs,m):
    k=0
    for p in axs.patches:
        k+=1
        if k==1:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[0]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==3:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[0]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==2:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[1]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==4:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[1]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
            

def percent4(axs,m):
    k=0
    for p in axs.patches:
        k+=1
        if k==1:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[1]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==3:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[3]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==2:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[2]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==4:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[4]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==5:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[1]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==6:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[2]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==7:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[3]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')
        if k==8:
            percentage = '{:.1f}%'.format(100 * (p.get_height()/df[m].value_counts()[4]))
            x = p.get_x() + p.get_width()/2
            y = p.get_height()/2
            axs.annotate(percentage, (x, y),ha='center')

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(15,10))
fig.suptitle('Visualising')

ax1 = sns.countplot(ax=axs[0,0],x="Accomodation_Type",hue='Response',data=df)
percent(ax1,"Accomodation_Type")

ax2 = sns.countplot(ax=axs[0,1],x="Reco_Insurance_Type",hue='Response', data=df)
percent(ax2,"Reco_Insurance_Type")

ax3 = sns.countplot(ax=axs[1,0],x="Is_Spouse",hue='Response', data=df)
percent(ax3,"Is_Spouse")

ax4=sns.countplot(ax=axs[1,1],x="Holding_Policy_Type",hue='Response', data=df)
percent4(ax4,"Holding_Policy_Type")



In [ ]:
fig,axs = plt.subplots(2,2,figsize=(16,10))
fig.suptitle('Visualising')

ax1 = sns.countplot(ax=axs[0,0],x='City_Code',hue='Response',data=df)
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=90)
ax2 = sns.countplot(ax=axs[0,1],x='Health Indicator',hue='Response', data=df)
ax3 = sns.countplot(ax=axs[1,0],x='Holding_Policy_Duration',hue='Response', data=df)
ax4=sns.countplot(ax=axs[1,1],x='Reco_Policy_Cat',hue='Response', data=df)

In [ ]:
def yn_ratio(data,column):
    dg = data[[column,'Response']].groupby([column])
    dg = dg.Response.value_counts().to_frame() 
    
    C,Yes,No=[],[],[]
    for i in range(len(dg)):
        if i%2!=0:
            Yes.append(dg.iloc[i,0])
        if i%2==0:
            No.append(dg.iloc[i,0])
            C.append(str(dg.index[i][0]))
            
    dic = {column:C,'Yes':Yes,'No':No}
    dgv = pd.DataFrame(dic)
    dgv['Y/N'] = dgv['Yes']/dgv['No']
 
    plt.figure(figsize=(14,6))
    ax = sns.barplot(dgv[column],dgv['Y/N'])
    plt.xticks(rotation=45)
    #return dgv

In [ ]:
yn_ratio(df,'City_Code')

In [ ]:
yn_ratio(df,'Health Indicator')

In [ ]:
yn_ratio(df,'Holding_Policy_Duration')

In [ ]:
yn_ratio(df,'Reco_Policy_Cat')

## Feature Enginnering

In [ ]:
df.columns

In [ ]:
#Test 1 : Averaging upper_age and lower_age
df1 = df.copy()
df1['mean_age'] = (df1.Upper_Age + df1.Lower_Age)/2
df1.head(1)

#result : same roc in all, so not advisable

In [ ]:
#Test 2: Changing Region code to category

df1 = df.copy()
df1['reg_code'] = df1['Region_Code'].apply(lambda x : 'A' if x<=1000 else
                                           ('B' if 1000<x<=2000 else
                                            ('C' if 2000<x<=3000 else ('D' if 3000<x<=4000 
                                                                       else ('E' if 4000<x<=5000 else 'F')))))
sns.countplot(df1['reg_code'],order=['A','B','C','D','E','F'])

#result1 : All 3 roc are lower than baseline, not advisable

In [ ]:
#Test 3 : Dropping Region Code
#result : roc dropped by 2-3%, Not advisable

In [ ]:
#Test 4 : Difference of Upper and Lower Age
df1 = df.copy()
df1['mean_age'] = df1.Upper_Age - df1.Lower_Age
df1.head(1)
#result : No such difference, not advisable

In [ ]:
sns.scatterplot(data=df, x='Response', y="Reco_Policy_Premium")

In [ ]:
#Test 5 : Removing outliers from Reco Policy Premium
df1 = df.copy()
df1 = df1.drop(df1.loc[df1.Reco_Policy_Premium>32000].index)
#result : roc improved by 1% , Advisable

In [ ]:
#Test 6 : New feature showing 1 if Health indicator =X7 or Reco policy cat = 15 or city_code=c30
df1 = df.copy()
nf = []
for i in range(len(df1)):
    if (df1['Health Indicator'].iloc[i]=='X7' or df1['Reco_Policy_Cat'].iloc[i]== 15 or df1['City_Code'].iloc[i]=='C30'):
        nf.append(1)
    else:
        nf.append(0)
df1['nf'] = nf
df1.loc[(df1['Health Indicator']=='X7') | (df1['Reco_Policy_Cat']== 15) | (df1['City_Code']=='C30')].head(2)

#result : Same, not advisable

In [ ]:
#Test7 : replacing 'Health Indicator','Holding_Policy_Duration','Reco_Policy_Cat' and 'City_Code' by their y/n percentage
df1 = df.copy()
dgv_cc = yn_ratio(df1,'City_Code').set_index('City_Code').to_dict()
dgv_hi = yn_ratio(df1,'Health Indicator').set_index('Health Indicator').to_dict()
dgv_hpd = yn_ratio(df1,'Holding_Policy_Duration').set_index('Holding_Policy_Duration').to_dict()
dgv_rpc = yn_ratio(df1,'Reco_Policy_Cat').set_index('Reco_Policy_Cat').to_dict()

cc,hi,hpd,rpc = [],[],[],[]
for i in range(len(df1)):
    city = df1.City_Code.iloc[i]
    yn_cc = (dgv_cc['Y/N'][city])
    cc.append(yn_cc)
    
    health = df1['Health Indicator'].iloc[i]
    yn_hi = (dgv_hi['Y/N'][health])
    hi.append(yn_hi)
    
    hold = df1.Holding_Policy_Duration.iloc[i]
    yn_hpd = (dgv_hpd['Y/N'][hold])
    hpd.append(yn_hpd)
    
    reco = df1.Reco_Policy_Cat.iloc[i]
    yn_rpc = (dgv_rpc['Y/N'][str(reco)])
    rpc.append(yn_rpc)
    
df1['cc'] = cc
df1['hi'] = hi
df1['hpd'] = hpd
df1['rpc'] = rpc

#result : roc declined , not advisable

In [ ]:
df1.head(2)

In [ ]:
#Test8 : Imputing missing values by machine learning
df1 = df.copy()
pd.isnull(df1).sum().sort_values(ascending=False)

#result : roc increased, advisable

In [ ]:
# Imputing Data using DeepLearning by Datawig library

def impute(data):
    col = ['Health Indicator','Holding_Policy_Duration','Holding_Policy_Type']
    for name in col:
        index_null=[]
        imputed_list = []
        completed = []
        input_col = [ 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse','Reco_Policy_Cat', 'Reco_Policy_Premium'] + completed
        
        for i in range(len(data)):
            if pd.isnull(data[name]).iloc[i]==True:
                index_null.append(i)
    
        index_nn = list(set(data.index)-set(index_null))

        df1_train, df1_test = data.loc[index_nn],data.loc[index_null]

        #Initialize a SimpleImputer model
        imputer = datawig.SimpleImputer(
            input_columns= input_col,output_column= name, output_path = 'imputer_model_'+name+'_test')

        #Fit an imputer model on the train data
        imputer.fit(train_df=df1_train)

        #Impute missing values and return original dataframe with predictions
        imputed = imputer.predict(df1_test)
        
        imputed_list = []
        for i in range(len(data)):
            if i%5000==0:
                print('J'+str(i))
            if pd.isnull(data[name].iloc[i])==True:
                idx = data.ID.iloc[i]
                val = imputed.loc[imputed.ID==idx][name+'_imputed'].iloc[0]
                imputed_list.append(val)
            else:
                val = data[name].iloc[i]
                imputed_list.append(val)
        data['imp_'+name] = imputed_list

        completed.append('imp_'+name)
        print('column {} is imputed with {} imputations'.format(name,len(imputed[name])))
        
        
# Generated three new_features imp_Health Indicator,imp_Holding_Policy_Duration,imp_Holding_Policy_Type which contains the values of 
#'Holding_Policy_Duration', 'Holding_Policy_Type' and 'Health Indicator' alongwith their imputed values. 
#data is stored in ../input/avjobhack/df1.csv

In [ ]:
df1 = pd.read_csv('../input/avjobhack/df1.csv')
pd.isnull(df1).sum().sort_values(ascending=False)

In [ ]:
# Test 9 : removing outliers from region_code
df1 = df.copy()
sns.boxplot(df1['Region_Code'])
df1 = df1.drop(df1.loc[df1.Region_Code>5800].index)

#result : roc decreased, not advisable

## Data Preparation

In [ ]:
test['Holding_Policy_Type'] = np.where(pd.isnull(test['Holding_Policy_Type']),test['Holding_Policy_Type'],test['Holding_Policy_Type'].astype(str))
impute(test)

# result is stored in ../input/avhacktest1/test_fin.csv

In [ ]:
test = pd.read_csv('../input/avhacktest1/test_fin.csv')

In [ ]:
test.head()

In [ ]:
train = pd.read_csv('../input/avjobhack/df1.csv')
train.info()

In [ ]:
train = train.drop(['Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type'],axis=1)
train = train.drop(train.loc[train.Reco_Policy_Premium>32000].index)
train.info()

In [ ]:
def data_prep(data):
    
    num_attribs = ['Region_Code','Upper_Age', 'Lower_Age','Reco_Policy_Premium']
    cat_attribs = ['City_Code','Accomodation_Type','Reco_Insurance_Type',
               'Is_Spouse','imp_Health Indicator','imp_Holding_Policy_Duration','imp_Holding_Policy_Type','Reco_Policy_Cat',]
    target = ['Response']
    
    print("num attribs :{}".format(num_attribs))
    print("cat attribs :{}".format(cat_attribs))
        
    X=data[num_attribs+cat_attribs]
    y=data[target]
    
    print('Data shape {} X shape {} y shape {} '.format(data.shape,X.shape,y.shape))

    full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(), cat_attribs)
    ],remainder='passthrough')
    
    X = full_pipeline.fit_transform(X) 
    
    return X,y 

X_train,y_train = data_prep(train)

In [ ]:
print(type(X_train.toarray()))
print(type(y_train))

In [ ]:
X_train.shape

## Modeling

### XGBoost

In [ ]:
#XGBoost
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn import metrics     #Additional scklearn functions
from sklearn.model_selection import GridSearchCV #Perforing grid search

train 
def modelfit(alg, X,y,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X, label=y.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(X, y,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(X)
    dtrain_predprob = alg.predict_proba(X)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(y.values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(y, dtrain_predprob))
      
    return alg     
    

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

model_xgb1 = modelfit(xgb1,X_train,y_train)

In [ ]:
model_xgb1

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = model_xgb1, 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[1,2,3,4,5,6,7]
}
gsearch2 = GridSearchCV(estimator = model_xgb1, 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
model_xgb1

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=237, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
model_xgb2 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=237, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.8,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = model_xgb2, 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
model_xgb3 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=237, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.9,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [ ]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = model_xgb3, 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train,y_train)
gsearch6.best_params_, gsearch6.best_score_

In [ ]:
model_xgb4 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, monotone_constraints='()',
              n_estimators=237, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=27, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=1, seed=27, subsample=0.9,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [ ]:
model_xgb4.fit(X_train,y_train)

In [ ]:
def pred(test,model):
    test = pd.read_csv('test_fin.csv')
    num_attribs = ['Region_Code','Upper_Age', 'Lower_Age','Reco_Policy_Premium']
    cat_attribs = ['City_Code','Accomodation_Type','Reco_Insurance_Type',
               'Is_Spouse','imp_Health Indicator','imp_Holding_Policy_Duration','imp_Holding_Policy_Type','Reco_Policy_Cat']    

        
    X=test[num_attribs+cat_attribs]

    
    print('Data shape {} X shape {}  '.format(test.shape,X.shape))

    full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
        ("cat", OneHotEncoder(), cat_attribs)
    ],remainder='passthrough')
    
    X_test = full_pipeline.fit_transform(X) 
    print(X_test.shape)
    
    y_fin = model.predict(X_test)
    
    return y_fin

In [ ]:
y_fin = pred(test,model_xgb4)

In [ ]:
y_fin

In [ ]:
fin = pd.DataFrame(y_fin, columns = ['Response'])
fin.head()

In [ ]:
fin.Response.value_counts()

In [ ]:
fin['ID'] = test['ID']
fin.head()

In [ ]:
fin.to_csv('fin_xgb2.csv',index=False)

### Random Forest Classifier

In [ ]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [50,100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10,15] 
max_features = [2, 3, 4 , 5,6,7,8]

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf, max_features=max_features)

forest = RandomForestClassifier()

gridF = RandomizedSearchCV(forest, hyperF, cv = 3, verbose = 1, n_jobs = -1)
bestF = gridF.fit(X_train,y_train)

In [ ]:
bestF.best_estimator_

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [40,45,50,55,60,65]
max_depth = [10,13,15,17,19]
min_samples_split = [2,3,1,4,5]
min_samples_leaf = [3,4, 5,7,9] 
max_features = [5,6,7,8,9,10,11,12,13]

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf, max_features=max_features)

forest = RandomForestClassifier()

gridF = RandomizedSearchCV(forest, hyperF, cv = 3, verbose = 1, n_jobs = -1)
bestF = gridF.fit(X_train,y_train)

In [ ]:
bestF.best_estimator_

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [55,60,65]
max_depth = [19,22,25]
min_samples_split = [5,8,10]
min_samples_leaf = [2,4,6] 
max_features = [13,17,20]

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf, max_features=max_features)

forest = RandomForestClassifier()

gridFin = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, n_jobs = -1)
bestF = gridFin.fit(X_train,y_train)

In [ ]:
RF = bestF.best_estimator_

In [ ]:
RF.fit(X_train,y_train)

In [ ]:
y_rf = pred(test,RF)

In [ ]:
fin = pd.DataFrame(y_rf, columns = ['Response'])
fin.head()

In [ ]:
fin['ID'] = test['ID']
fin.Response.value_counts()

In [ ]:
fin.to_csv('fin_rf.csv',index=False)

### CatBoost Classifier

A little story before we move on to this part, the Catboost classifier part was added after the competition to the notebook. I tried XGBoost, Random Forest but was stuck in 0.68 ROC-AUC. I even tried in another google colab notebook parallely Light GBM and KerasClassifier(Neural Network) but alas none could cross the 0.70 mark.

But then today in the competition forum discussion part I saw Shobhit saying a magic word can increase your score to 0.80 and then I applied CatBoost. What to do sometimes, luck is not on your side :)

In [ ]:
# Credit for below part : shobhit upadhyaya ,Github link : https://github.com/asingleneuron

In [ ]:
df1 = pd.read_csv('../input/avjobhack/df1.csv')
test = pd.read_csv('../input/avhacktest1/test_fin.csv')
df1 = df1.drop(df1.loc[df1.Reco_Policy_Premium>32000].index)

In [ ]:
df1['imp_Holding_Policy_Type'] = df1['imp_Holding_Policy_Type'].astype(str)
test['imp_Holding_Policy_Type'] = test['imp_Holding_Policy_Type'].astype(str)

df1['Reco_Policy_Cat'] = df1['Reco_Policy_Cat'].astype(str)
test['Reco_Policy_Cat'] = test['Reco_Policy_Cat'].astype(str)

df1['Region_Code'] = df1['Region_Code'].astype(str)
test['Region_Code'] = test['Region_Code'].astype(str)

In [ ]:
cols_to_remove = ['ID','Health Indicator','Holding_Policy_Duration','Holding_Policy_Type']
target = 'Response'

_X = df1.drop(cols_to_remove + [target], axis=1)
y = df1[target]
_XTEST = test.drop(cols_to_remove , axis=1)

In [ ]:
_X.shape, _XTEST.shape

In [ ]:
X_all = pd.concat([_X, _XTEST]).reset_index(drop=True)
X_all.shape

In [ ]:
X_all.info()

In [ ]:
cat_columns = []
for col in X_all.select_dtypes('object').columns:
    print(col)
    cat_columns.append(col)
    le = LabelEncoder()
    X_all[col] = le.fit_transform(X_all[col])

In [ ]:
X = X_all[:len(y)]
XTEST = X_all[len(y):]
X.shape, XTEST.shape

In [ ]:
NUM_OF_BOOST_ROUND = 10000
EARLY_STOPPING = 300

In [ ]:
cat_features_index = [i for i,col in enumerate(X.columns) if col in cat_columns]
cat_features_index

In [ ]:
X.columns

In [ ]:
X_train , X_valid, y_train, y_valid = train_test_split(X,y, 
                                                       test_size=0.2, 
                                                       random_state=56, 
                                                       stratify=y)

In [ ]:
params = {
    'cat_features': cat_features_index,
    'eval_metric': 'AUC',
    'random_seed': 56,
    'n_estimators': NUM_OF_BOOST_ROUND,
}

In [ ]:
bst = CatBoostClassifier(**params, early_stopping_rounds=EARLY_STOPPING)
_ = bst.fit(X_train, y_train, eval_set=(X_valid,y_valid), plot=True, verbose=False)

In [ ]:
ypred_cat = bst.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid, ypred_cat)

In [ ]:
f_importance_df = pd.DataFrame(bst.get_feature_importance(), columns=['importance'], index=X_valid.columns)
f_importance_df = f_importance_df.sort_values(by='importance', ascending=False)

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=f_importance_df.importance[:500], y=f_importance_df.index[:500]);

In [ ]:
ypred_test = bst.predict_proba(XTEST)[:,1]
pred = pd.DataFrame(ypred_test)
pred.to_csv('pred.csv')

In [ ]:
sns.distplot(ypred_test)